In [ ]:
import torch
import torchvision.transforms as tfms

import os, math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MAP_NAME = 'map_64x64'

with open(f'./data/{MAP_NAME}.txt') as file:
    vals = file.readline().rstrip().rsplit(' ')                 # gets string array of [size_x, size_y]
    size_x, size_y = int(vals[0]), int(vals[1])

MAP_DIMS = (size_x,size_y)
START = (10,10)
GOAL = (55,55)
NUM_PATHS = 100
STEP_DIS = 1

transforms = tfms.Compose(
    [
        tfms.ToTensor(),
        tfms.Normalize([0.5 for _ in range(1)], [0.5 for _ in range(1)])
    ]
)

In [ ]:
class PathsDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform=None, shape = (100,100)):
        self.paths = [] # create a list to hold all paths read from file
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
                self.flat_path = np.loadtxt(f) # load in the flat path from file
                self.path = np.asarray(self.flat_path, dtype=int).reshape(len(self.flat_path)//2,2) #unflatten the path from the file

                self.path_matrix = self.convert_path(shape, self.path)
                
                self.paths.append(self.path_matrix) # add the path to paths list
        self.transform = transform
        print("Done!")

    def convert_path(self, map_dim, path):
        path_mat = np.zeros(map_dim, dtype=float)

        # Make the path continuous
        for i in range(path.shape[0] - 1):
            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            if (x1 < x2):
                x_dir = 1
            else:
                x_dir = -1

            if (y1 < y2):
                y_dir = 1
            else:
                y_dir = -1

            # Determine y from x
            if x2 != x1:
                m = (y2-y1)/(x2-x1)
                while x != x2:
                    y = round(m*(x-x1) + y1)
                    path_mat[y,x] = 1
                    x += x_dir
            else:
                while x != x2:
                    path_mat[y1,x] = 1
                    x += x_dir


            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            # Determine x from y
            if y2 != y1:
                m = (x2-x1)/(y2-y1)
                while y != y2:
                    x = round(m*(y-y1) + x1)
                    path_mat[y,x] = 1
                    y += y_dir
            else:
                while y != y2:
                    path_mat[y,x1] = 1
                    y += y_dir
            
        path_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1     # Include the last point in the path

        return path_mat

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        x = np.float32(self.paths[idx])

        if self.transform:
            x = self.transform(x).cuda()

        return x

In [ ]:
def sort_data(path):

    sorted_path = [[START[0], START[1]]]              # ensure start point is proper path start point
    remaining_points = [[x,y] for x,y in path]
    remaining_points.append([GOAL[0], GOAL[1]])

    sorted_path_len = len(sorted_path)
    while sorted_path[sorted_path_len-1] != [GOAL[0], GOAL[1]]:
        prev_x, prev_y = sorted_path[sorted_path_len-1][0], sorted_path[sorted_path_len-1][1]
        next_point_idx = None
        smallest_dis = None
        
        for i in range(len(remaining_points)):
            curr_x, curr_y = remaining_points[i][0], remaining_points[i][1]
            dis = math.sqrt((prev_x-curr_x)**2 + (prev_y-curr_y)**2)

            if smallest_dis == None or smallest_dis > dis:
                smallest_dis = dis
                next_point_idx = i

        if smallest_dis != 0:
            sorted_path.append(remaining_points[next_point_idx])
        remaining_points.pop(next_point_idx)

        sorted_path_len = len(sorted_path)

    return np.asarray(sorted_path)

In [ ]:
def smoothen(path, loops):
    new_path = path.astype(float)

    for _ in range(loops):
        for i in range(1, len(path)-1):
            prev_x, prev_y = path[i-1][0], path[i-1][1]
            curr_x, curr_y = path[i][0], path[i][1]
            next_x, next_y = path[i+1][0], path[i+1][1]

            new_path[i] = [(prev_x+curr_x+next_x)/3.0, (prev_y+curr_y+next_y)/3.0]
            
        path = new_path

    return new_path

In [ ]:
# open and unflatten map file:
flat_map = np.loadtxt(f'./data/{MAP_NAME}.txt', skiprows=2)             
map = np.asarray(flat_map).reshape(MAP_DIMS[0], MAP_DIMS[1])

In [ ]:
# open and unflatten path files:
paths = []
for i in range(NUM_PATHS):
    flat_path = np.loadtxt(f'./data/vegan_paths/path_{i}_.txt', delimiter=',')             
    path = np.asarray(flat_path)
    paths.append(path)

In [ ]:
dir_name = "paths/"
if not os.path.exists(dir_name):                                    # if path folder for this map doesn't exist, create it
    os.mkdir(dir_name)
dir_name = f"{dir_name}{MAP_NAME}/"
if not os.path.exists(dir_name):                                    # if path folder for this map doesn't exist, create it
    os.mkdir(dir_name)

for i in range(NUM_PATHS):

    # getting path data from GAN output:
    paths[i] = np.round(paths[i])
    path_coords = np.argwhere(paths[i]==1)
    path_coords = sort_data(path_coords)
    new_path_coords = smoothen(path_coords, 5)

    # displaying path data:
    plt.figure(figsize=(7.5,7.5))
    plt.imshow(paths[i])
    plt.plot(path_coords[:, 1], path_coords[:, 0], c='r')
    plt.plot(new_path_coords[:, 1], new_path_coords[:, 0], c='g', linewidth=5)
    plt.show()

    # flatten + write path to file:
    flat_path = new_path_coords.flatten()      # flipped so start point is at front of file, end point is at end of file
    np.savetxt(f"{dir_name}path_{i}.txt", flat_path, fmt='%d') 